In [45]:
# import required modules
import requests
from bs4 import BeautifulSoup
import pandas as pd
import geopy
import os
import re
from geopy.geocoders import Nominatim

In [22]:
# get URL
page = requests.get("https://en.wikipedia.org/wiki/List_of_rugby_union_competitions")
soup = BeautifulSoup(page.content, 'html.parser')

In [209]:
years = [year for year in range(2021,2023)]
results = []
for year in years:
    results.append(result_scrapping(year, 'July_rugby_union_tests'))
results

An error occurred: list index out of range 2021 July_rugby_union_tests
An error occurred: list index out of range 2021 July_rugby_union_tests
An error occurred: list index out of range 2021 July_rugby_union_tests


[[['Zimbabwe_v_Zambia', '2 June 2021', '31–8', 'Police Grounds, Harare[4]'],
  ['Zimbabwe_v_Zambia', '5 June 2021', '56–3', 'Police Grounds, Harare[4]'],
  ['Japan XV_v_', '12 June 2021', '32–17', 'Shizuoka Stadium, Shizuoka[5]'],
  ['Māori All Blacks_v_Samoa',
   '26 June 2021',
   '35–10',
   'Wellington Regional Stadium, Wellington[8]'],
  ['British & Irish Lions_v_Japan',
   '26 June 2021',
   '28–10',
   'Murrayfield Stadium, Edinburgh[9]'],
  ['England A_v_Scotland A',
   '27 June 2021',
   'Cancelled[e]',
   'Welford Road Stadium, Leicester[14]'],
  ['South Africa_v_Georgia',
   '2 July 2021',
   '40–9',
   'Loftus Versfeld Stadium, Pretoria[15]'],
  ['Māori All Blacks_v_Samoa',
   '3 July 2021',
   '38–21',
   'Mount Smart Stadium, Auckland[16]'],
  ['New Zealand_v_Tonga',
   '3 July 2021',
   '102–0',
   'Mount Smart Stadium, Auckland[16]'],
  ['Ireland_v_Japan', '3 July 2021', '39–31', 'Aviva Stadium, Dublin'],
  ['Wales_v_Canada', '3 July 2021', '68–12', 'Millennium Stadium,

In [199]:
url = 'https://en.wikipedia.org/wiki/{}_mid-year_rugby_union_tests'


In [210]:
tournament_year = {}
tournament_year['mid-year_rugby_union_tests'] = [year for year in range(2004,2011)]
tournament_year['June_rugby_union_tests'] = [year for year in range(2013,2019)]
tournament_year['July_rugby_union_tests'] = [year for year in range(2021,2023)]

tournament_year['Rugby_World_Cup'] = [1987, 1991, 1995, 1999,2003,207,2011,2015,2019, 23]
tournament_year['Home_Nations_Championship'] = [year for year in range(1883,1910)]
tournament_year['Five_Nations_Championship'] = [year for year in range(1910,1915)] +  [year for year in range(1920,1932)]
tournament_year['Home_Nations_Championship'] = [year for year in range(1932,1940)]
tournament_year['Five_Nations_Championship'] = [year for year in range(1940,2000)]
tournament_year['Six_Nations_Championship'] = [year for year in range(2000,2023)]
tournament_year['Tri_Nations_Series'] = [year for year in range(1996,2012)]
tournament_year['Rugby_Championship'] = [year for year in range(2012,2024)]
tournament_year['Rugby_Union_European_Cup'] = [1952, 1954]
tournament_year['FIRA_Nations_Cup'] = [f"{year}-{year+1}" for year in range(1965, 1973)]
tournament_year['FIRA_Trophy'] =  [f"{year}-{year+1}" for year in range(1973, 1995)]
tournament_year['FIRA_Tournament'] = ['1996-1997']
tournament_year['European_Nations_Cup_First_Division'] = [2000,2001]+[year for year in range(2010,2017)] + ['2001-2002', '2003-2004', '2004-2006', '2006-2008', '2008-2010']
tournament_year['Rugby_Europe_International_Championships'] = [f"{year}-{year+1}" for year in range(2016, 2024)]
tournament_year['IRB_Pacific_5_Nations'] = [year for year in range(2006,2015)]
tournament_year['World_Rugby_Pacific_Nations_Cup']= [year for year in range(2015,2024)]
tournament_year['IRB_Nations_Cup']= [year for year in range(2006,2015)]
tournament_year['World_Rugby_Nations_Cup']= [year for year in range(2015,2020)]
tournament_year['Americas_Rugby_Championship']= [year for year in range(2009,2020)]
tournament_year['ARFU_Asian_Rugby_Championship'] = [year for year in range(1969,1997)]+[1998,2000,2002,2004]+['2007-2008']+[year for year in range(2008,2024)]
tournament_year['South_American_Rugby_Championship_"A"'] = [year for year in range(1951,2018)]
tournament_year['South_American_Six_Nations'] = [2018]

In [254]:
#import the file from rugbycology, and add features city, venue and country,to make it coherent with the scrap data

current_dir = os.getcwd()
folder = '/data/'
file = 'all_games_RWC2023_fixed.csv'
tournaments_url = 'https://en.wikipedia.org/wiki/List_of_rugby_union_competitions'
result_folder = 'data/results_international/'

def switch_home(row):
    #tinput: a row of the rwc2023 dataset. 
    #output: a row with the inverted columns
    #function: if the team B is NZ(home team), switch teams A and B as well as scores A and B
    if row['Team B'] == 'New Zealand':
        row['Team A'], row['Team B'] = row['Team B'], row['Team A']
        row['Score A'], row['Score B'] = row['Score B'], row['Score A']
    if row['Team A'] == 'New Zealand':
        row['Neut.'] = False
    return row
    
def import_wc2023(file): 
    #input: the rwx2023 file
    #output: the imported filed with added columns
    #function: selcect only the necessary columns, and add Neut., city, Tourn, Venue and country variables
    df= pd.read_csv(file)
    df = df[['Date', 'Team A', 'Team B', 'Score A', 'Score B']]
    df['City'] = 'Auckland'
    df['Neut.'] = True
    df = df.apply(switch_home, axis=1)
    df['Tourn'] = 'WC'
    df['Venue'] = 'Na'
    df['Country'] = 'New Zealand'
    df['Date'] = df['Date'].apply(lambda date: pd.to_datetime(date, dayfirst=True, format='%d-%b-%Y'))
    return df

def find_tournament_list(link):
    #input: the wiki page with the international rugby competition
    #output: a list of the tournaments
    #function: used to obtain missing results after 2021
    page = requests.get(link)
    soup = BeautifulSoup(page.content, 'html.parser')
    tournament_links = []
    tournament_table = soup.find_all('table')[0]
    tournament_row = tournament_table.find_all('tr')
    
    for tournament in tournament_row:
        tf = tournament.find('td')
        a = tournament.find('a',href=True)
        if a:
            if 'Women' not in a['href']:
                tournament_links.append(a['href'][6:])
    return tournament_links

def result_scrapping(year, url):
    #input: a year and the championship partial url
    #output: A list with a match result: teams, score, venue, city
    #function: will be used to collect data for match since 2021. Can be Updated to collect more data if necessary
    try:
        page = requests.get(f"https://en.wikipedia.org/wiki/{year}_{url}", allow_redirects=False, timeout=5)
        results = []
        if page.status_code == 200:
            soup = BeautifulSoup(page.content, 'html.parser')
            fixtures =  soup.find_all(class_='vevent summary')
            
            for i in range(len(fixtures)):
                try:
                    tables = fixtures[i].find_all('table')
                    teams = tables[1].find_all('td', class_='vcard')
                    team_a = teams[0].find('a').text
                    team_b = teams[1].find('a').text
            
                    info = fixtures[i].find('tr')
                    td = info.find('td')
                    td.select("br")[0].replace_with("\n")
                    date= td.text.split('\n')[0]
                    #remove trailing info
                    date_str = date[:date.rindex(f'{year}') + len(f'{year}')]
                    #find the score
                    table_result = tables[1]
                    result = table_result.find_all('td')[1].text
                    #find the venue
                    venue = tables[2]
                    venue.select("br")[0].replace_with("\n")
                    venue = venue.text.split('\n')[0]
                    results.append([f'{team_a}_v_{team_b}', date_str, result, venue])
                except Exception as e:
                    print(f"An error occurred: {e} {year} {url}")
                    continue
    except Exception as e:
        print(f"An error occurred while fetching page: {e}")
    
    return results

def scrappe_match_results(urls, years):
    #input: a list of championship url, and the years we want to collect data from
    #output: a dict with championship as key and list(year) of list of results
    #function: main function, collect data from wikipedia
    scapped_results = {}
    for url in urls[1:]:
        #here do not select the first value in url. its world cup and import rugby league world cup 2022. no bueno. requires more fine tuning
        tournament_results = [result_scrapping(years[i], url) for i in range(len(years))] 
        scapped_results[url]=tournament_results
    return scapped_results
    
def find_country(city, geo):
    #inputs: a string with a city name, a Nominatim object
    #output: the country of the city
    #function: will help define if a team is playing at home
    location = geo.geocode(city, language='en')
    if location:

        country_name = location.raw['display_name'].split(', ')[-1]
        if country_name == 'United Kingdom':
            if location.raw['display_name'].split(', ')[-2] != 'England':
                country_name = location.raw['display_name'].split(', ')[1]
            else:
                country_name = 'England'
        return country_name
    return None

def create_result_df(list_results, geo):
    #input: take dict with champiship as key and lists with results for each years as entry, a nominatim object
    #output: a dataframe 
    #function: creates a df with similar entries as rwc2023, will be used to update elo and underdog study
    df = pd.DataFrame(columns=['Date', 'Team A','Team B','Score A','Score B', 'Tourn','Venue', 'City', 'Neut.'])
    #access each tournament
    dates = []
    home_score = []
    away_score = []
    home_team = []
    away_team = []
    competition = []
    stadium = []
    city = []
    for entry in list_results:
        tournaments = list_results[entry]
        #access each year
        for i in range(len(tournaments)):
            #access each games
            try:
                for j in range(len(tournaments[i])):
                    score = tournaments[i][j][2]
                    scores = re.split('–|-', score)
                    if len(scores) != 2:
                        print(f"Entry at index {j} only contains a date.")
                        print(tournaments[i][j])
                        pass
                    
                    dates.append(tournaments[i][j][1])
                    home_score.append(int(scores[0]))
                    away_score.append(int(scores[1]))
                    teams = tournaments[i][j][0].split('_v_')
                    home_team.append(teams[0])
                    away_team.append(teams[1])
                    competition.append(entry)
                    geo = tournaments[i][j][-1].split(', ')
                    stadium.append(geo[0])
                    city.append(geo[1])
            except:
                pass
    #make sure we have a datetime object
    print(len(dates),len(home_score), len(away_score),len(competition),len(stadium),len(city)  )
    date_parsed = [pd.to_datetime(date,dayfirst=True, format='%d %B %Y') for date in dates]
    df['Date']= date_parsed
    df['Score A']= home_score
    df['Score B']= away_score
    df['Team A']= home_team
    df['Team B']= away_team
    df['Tourn']= competition
    df['Venue']= stadium
    df['City'] = city
    df['Country'] = df.apply(lambda row : find_country(row['City'], geolocator), axis=1 )
    # Add the neutrtal when necessary
    df['Neut.'] = df.apply(lambda row: row['Team A']!=row['Country'], axis=1)
    return df

def scrap_slams(url):
    #input: the url to the grand slam page. 
    #output: a list of results, for each year, containing only teams, scores, and year
    #function: use in conjunction with df_from slam to create a df with the results, that has the feature as the other df
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')   
    grand_slam = []
    games = []
    tournament_table = soup.find_all('table')[5]
    tournament_body = tournament_table.find('tbody')
    tournament_rows = (tournament_body.find_all('tr')[2:8] +
                       tournament_body.find_all('tr')[9:-2])
    
    for row in tournament_rows:
        tds = row.find_all('td')
        years = tds[0].text.strip()
        if years == 'South Africa':
            years = '2010'
            away = tds[1].text.strip()
            england = tds[3].text.strip()
            ireland = tds[4].text.strip()
            scotland = tds[5].text.strip()
            wales = tds[6].text.strip()
        else:
            away = tds[1].text.strip()
            england = tds[4].text.strip()
            ireland = tds[5].text.strip()
            scotland = tds[6].text.strip()
            wales = tds[7].text.strip()
        games.append([years, away, england, ireland,scotland, wales])
    return games
    
def df_from_slams(lists):
    #input: a list of list created by scrap slams
    #output: a df with the values
    #function: collect info from slams
    df = pd.DataFrame(columns=['Date', 'Team A','Team B','Score A','Score B', 'Tourn','Venue', 'City', 'Neut.'])
    dates = []
    home_score = []
    away_score = []
    home_team = []
    away_team = []
    competition = []
    stadium = []
    city = []
    for list_ in lists:
        for i in range(4):
            dates.append(list_[0].split('–')[0])
        score1 = list_[2].split('–')
        score2 = list_[3].split('–')
        score3 = list_[4].split('–')
        score4 = list_[5].split('–')
        for i in range(4):
            away_team.append(list_[1])
        home_team += ['England', 'Ireland', 'Scotland', 'Wales']
        home_score.extend([score1[1],score2[1],score3[1], score4[1]])
        away_score.extend([score1[0],score2[0],score3[0], score4[0]])
    df['Date'] = dates
    df['Team A'] = home_team
    df['Team B'] = away_team
    df['Score A'] = home_score
    df['Score B'] = away_score
    df['Tourn'] = 'Grand Slam'
    df['Neut.'] = False
    return df

def df_slams():
    #creates a slam df with results
    grand_slam_link = "https://en.wikipedia.org/wiki/Grand_Slam_(rugby_union)#Grand_Slam_tours"
    return df_from_slams(scrap_slams(grand_slam_link))

def df_master(tournament_dict):
    #create a df from all games from all tournaments
    scrapped_results = {}
    for tourn, years in tournament_dict.items():
        partial_result = []
        for year in years:
            partial_result.append(result_scrapping(year, tourn))
        scrapped_results[tourn] = partial_result
    return scrapped_results
    

In [217]:
result = df_master(tournament_year)

An error occurred: list index out of range 2004 mid-year_rugby_union_tests
An error occurred: list index out of range 2004 mid-year_rugby_union_tests
An error occurred: list index out of range 2005 mid-year_rugby_union_tests
An error occurred: list index out of range 2005 mid-year_rugby_union_tests
An error occurred: list index out of range 2007 mid-year_rugby_union_tests
An error occurred: list index out of range 2007 mid-year_rugby_union_tests
An error occurred: list index out of range 2008 mid-year_rugby_union_tests
An error occurred: list index out of range 2008 mid-year_rugby_union_tests
An error occurred: 'NoneType' object has no attribute 'text' 2010 mid-year_rugby_union_tests
An error occurred: 'NoneType' object has no attribute 'text' 2010 mid-year_rugby_union_tests
An error occurred: 'NoneType' object has no attribute 'text' 2014 June_rugby_union_tests
An error occurred: list index out of range 2021 July_rugby_union_tests
An error occurred: list index out of range 2021 July_r

In [255]:
result.keys()
geolocator = Nominatim(user_agent = "AGexercice", timeout=10)
df = create_result_df(result, geolocator)

Entry at index 5 only contains a date.
['England A_v_Scotland A', '27 June 2021', 'Cancelled[e]', 'Welford Road Stadium, Leicester[14]']
Entry at index 8 only contains a date.
['France_v_Scotland', '28 February 2021', 'Postponed[b]', 'Stade de France, Saint-Denis']
1098 1095 1095 1095 1095 1091


ValueError: Length of values (1095) does not match length of index (1098)

In [224]:
df

,Date,Team A,Team B,Score A,Score B,Tourn,Venue,City,Neut.,Country
0,2021-02-06,Italy,France,10,50,Six_Nations_Championship,Stadio Olimpico,Rome,False,Italy
1,2021-02-06,England,Scotland,6,11,Six_Nations_Championship,Twickenham Stadium,London,False,England
2,2021-02-07,Wales,Ireland,21,16,Six_Nations_Championship,Millennium Stadium,Cardiff,False,Wales
3,2021-02-13,England,Italy,41,18,Six_Nations_Championship,Twickenham Stadium,London,False,England
4,2021-02-13,Scotland,Wales,24,25,Six_Nations_Championship,Murrayfield Stadium,Edinburgh,False,Scotland
...,...,...,...,...,...,...,...,...,...,...
94,2022-11-22,Nepal,Bangladesh,12,43,Asia_Rugby_Championship,Rabindra Sarobar Stadium,Kolkata,True,India
95,2022-11-25,India,Bangladesh,82,0,Asia_Rugby_Championship,Rabindra Sarobar Stadium,Kolkata,False,India
96,2023-06-03,Malaysia,South Korea,3,27,Asia_Rugby_Championship,Siadinha Hamzah Stadium,Kuala Lumpur,False,Malaysia
97,2023-06-10,Hong Kong,Malaysia,88,9,Asia_Rugby_Championship,Hong Kong Football Club Stadium,Hong Kong,True,China


In [220]:
def create_result_df(result_dict, geo):
    #input: take dict with champiship as key and lists with results for each years as entry, a nominatim object
    #output: a dataframe 
    #function: creates a df with similar entries as rwc2023, will be used to update elo and underdog study
    df = pd.DataFrame(columns=['Date', 'Team A','Team B','Score A','Score B', 'Tourn','Venue', 'City', 'Neut.'])
    #access each tournament
    dates = []
    home_score = []
    away_score = []
    home_team = []
    away_team = []
    competition = []
    stadium = []
    city = []
    for entry in scapped_results:
        tournaments = scapped_results[entry]
        #access each year
        for i in range(len(tournaments)):
            #access each games
            for j in range(len(tournaments[i])):
                score = tournaments[i][j][2]
                scores = re.split('–|-', score)
                if scores[0] != 'Postponed[b]':
                    dates.append(tournaments[i][j][1])
                    
                    home_score.append(int(scores[0]))
                    away_score.append(int(scores[1]))
                    teams = tournaments[i][j][0].split('_v_')
                    home_team.append(teams[0])
                    away_team.append(teams[1])
                    competition.append(entry)
                    geo = tournaments[i][j][-1].split(', ')
                    stadium.append(geo[0])
                    city.append(geo[1])
    #make sure we have a datetime object
    date_parsed = [pd.to_datetime(date,dayfirst=True, format='%d %B %Y') for date in dates]
    df['Date']= date_parsed
    df['Score A']= home_score
    df['Score B']= away_score
    df['Team A']= home_team
    df['Team B']= away_team
    df['Tourn']= competition
    df['Venue']= stadium
    df['City'] = city
    df['Country'] = df.apply(lambda row : find_country(row['City'], geolocator), axis=1 )
    # Add the neutrtal when necessary
    df['Neut.'] = df.apply(lambda row: row['Team A']!=row['Country'], axis=1)
    return df


dict_keys(['mid-year_rugby_union_tests', 'June_rugby_union_tests', 'July_rugby_union_tests', 'Rugby_World_Cup', 'Home_Nations_Championship', 'Five_Nations_Championship', 'Six_Nations_Championship', 'Tri_Nations_Series', 'Rugby_Championship', 'Rugby_Union_European_Cup', 'FIRA_Nations_Cup', 'FIRA_Trophy', 'FIRA_Tournament', 'European_Nations_Cup_First_Division', 'Rugby_Europe_International_Championships', 'IRB_Pacific_5_Nations', 'World_Rugby_Pacific_Nations_Cup', 'IRB_Nations_Cup', 'World_Rugby_Nations_Cup', 'Americas_Rugby_Championship', 'ARFU_Asian_Rugby_Championship', 'South_American_Rugby_Championship_"A"', 'South_American_Six_Nations'])

In [60]:
#perform the importation of data from wikipedia
start = 2021
end = 2024
wanted_period = [year for year in range(start, end)]
#import rwc2023 data
rwc2023_results_filtered = import_wc2023(current_dir+folder+file)
tournament_links = find_tournament_list(tournaments_url)
scrapped_result = scrappe_match_results(tournament_links, wanted_period)
geolocator = Nominatim(user_agent = "AGexercice", timeout=10)
#scrapp missing data
result_df= create_result_df(scrapped_result,geolocator)
#concat and export to xlsx
final_result = pd.concat([result_df, rwc2023_results_filtered], ignore_index=True)
final_result.to_excel(result_folder+'scrap_out.xlsx')
final_result.head(5)

An error occurred: list index out of range 2022 Asia_Rugby_Championship
An error occurred: list index out of range 2022 Asia_Rugby_Championship
An error occurred: substring not found 2022 Asia_Rugby_Championship


,Date,Team A,Team B,Score A,Score B,Tourn,Venue,City,Neut.,Country
0,2021-02-06,Italy,France,10,50,Six_Nations_Championship,Stadio Olimpico,Rome,False,Italy
1,2021-02-06,England,Scotland,6,11,Six_Nations_Championship,Twickenham Stadium,London,False,England
2,2021-02-07,Wales,Ireland,21,16,Six_Nations_Championship,Millennium Stadium,Cardiff,False,Wales
3,2021-02-13,England,Italy,41,18,Six_Nations_Championship,Twickenham Stadium,London,False,England
4,2021-02-13,Scotland,Wales,24,25,Six_Nations_Championship,Murrayfield Stadium,Edinburgh,False,Scotland
